Example stadincijfers usage in python
=========

Make sure that stadincijfers is installed.
If needed, run the following in terminal:

    pip install stadincijfers


In [5]:
import pandas as pd
import os, sys

In [2]:
from stadincijfers import stadincijfers

In [3]:
sic = stadincijfers("antwerpen")

### Geolevels and periodlevels.

You can get a dictionary of available periodlevels and geolevels on the server by calling `.geolevels()` or `.periodlevels()` on the sic-object. 

In [5]:
sic.periodlevels()

{'halfyear': 'Halfjaar',
 'month': 'Maand',
 'quarter': 'Kwartaal',
 'year': 'Jaar'}

### Getting the lists of available data

You can also get the open available variables (cijfers) and their name, but this can take a while to read them all. These servers seem to have a lot of data. 

You need to give a starting en endpoint for this tool to loop through, they will be fetched in blocks of 10. Something like `sic.odataVariables(0, 100000)` will get them all.

Best to save your results to a file, so won't need to fetch them every time. 

In [8]:
vars = sic.odataVariables(16700, 17000)
vars_s = pd.Series(vars)
vars_s.to_csv("data/vars.csv")
vars_s.tail()

reading data, lines 16700 to 17000 this can take a while
16700 16710 16720 16730 16740 16750 16760 16770 16780 16790 16800 16810 16820 16830 16840 16850 16860 16870 16880 16890 

ziekteverz_tot_index    Groeicijfer leerkrachten basisonderwijs met zi...
ziekteverz_tot_m        Aantal mannelijke leerkrachten basisonderwijs ...
ziekteverz_tot_v        Aantal vrouwelijke leerkrachten basisonderwijs...
zonnbp                      Aantal ontvangers van een zonne-energiepremie
zorgindex_familie                                     Familiale zorgindex
dtype: object

### Getting the data 

If have a variable from the odataVariables-list you want to explore, you can fetch them with `selectiontableasjson` as a json-object or `selectiontableasDataframe` as a pandas-dataframe. You also to specify **geolevel** , **periodlevel** and which **period**, usually a year, you want to select data for. 

In [7]:
# get data as a json object convert to dict
v = sic.selectiontableasjson('autodeelplaatsen_cambio', geolevel='sector', periodlevel='year', period=2020 )
v['rows'][0:10]

[['12MQ', '2020', ''],
 ['A00-', '2020', '2'],
 ['A01-', '2020', ''],
 ['A02-', '2020', '9'],
 ['A03-', '2020', ''],
 ['A04-', '2020', '3'],
 ['A05-', '2020', '2'],
 ['A081', '2020', ''],
 ['A10-', '2020', ''],
 ['A11-', '2020', '5']]

In [21]:
cambio2016 = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='wijk', periodlevel='year', period=2016)
cambio2016.head(10)


,Geo,Perioden,# autodeelplaatsen Cambio
0,Amandus - Atheneum,2016,2.0
1,Brederode,2016,NaN
2,Centraal Station,2016,NaN
3,Dam,2016,NaN
4,Eilandje,2016,2.0
5,Haringrode,2016,2.0
6,Harmonie,2016,7.0
7,Haven Antwerpen,2016,NaN
8,Historisch Centrum,2016,4.0
9,Kiel,2016,NaN


### Doing calculations

In [20]:
cambioNu = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='wijk', periodlevel='year', period=2020)
"cambios staansplaatsen in 2016 {} en nu {}".format( cambio2016['# autodeelplaatsen Cambio'].sum() ,
                                            cambioNu['# autodeelplaatsen Cambio'].sum() ) 

'cambios in 2016 74.0 en nu 156.0'